In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import math


In [41]:

L1 = [np.random.choice([1, -1]) for i in range (25)]
L1 = np.array(L1)
L1 = L1.reshape(5,5)
L1[L1 == 1] = 90
L1[L1 == -1] = 270

def Hamiltonian2D(J, L, x , y):
    n = np.shape(L)[0]
    Hamiltonian2D = -J * ((np.cos(L[x, y] - L[x, (y+1)%n])) + (np.cos(L[x, y] - L[x, (y-1)%n])) + (np.cos(L[x, y] - L[(x+1)%n, y])) + (np.cos(L[x, y] - L[(x-1)%n, y])))
    return np.array(Hamiltonian2D)

print(Hamiltonian2D(1, L1, 1, 1))


2.3938402762314324


In [46]:
# Hamiltonian array

x, y = np.indices(L1.shape)
L_hamiltonian = Hamiltonian2D(1,L1, x,y)
print (L_hamiltonian)

[[-4.         -2.40153993 -2.40153993 -0.80307986 -2.40153993]
 [-0.80307986  2.39384028 -0.80307986 -0.80307986 -0.80307986]
 [-2.40153993 -2.40153993 -2.40153993 -0.80307986 -2.40153993]
 [-2.40153993 -4.         -2.40153993  0.79538021  0.79538021]
 [-4.         -2.40153993  0.79538021  0.79538021 -0.80307986]]


In [42]:
def P_dot_i_2D(J, L, x , y):
    n = np.shape(L)[0]
    P_dot_i_2D = J * ((np.sin(L[x, y] - L[x, (y+1)%n])) + (np.sin(L[x, y] - L[x, (y-1)%n])) + (np.sin(L[x, y] - L[(x+1)%n, y])) + (np.sin(L[x, y] - L[(x-1)%n, y])))
    return np.array(P_dot_i_2D)

print(P_dot_i_2D(1, L1, 1, 1))

-3.2046105429353218


In [47]:
# P_Dot_i array


x, y = np.indices(L1.shape)
L_P_dot_i = P_dot_i_2D(1,L1, x,y)
print (L_P_dot_i)

[[ 0.          0.80115264  0.80115264  1.60230527  0.80115264]
 [ 1.60230527 -3.20461054  1.60230527 -1.60230527 -1.60230527]
 [ 0.80115264  0.80115264  0.80115264 -1.60230527 -0.80115264]
 [ 0.80115264  0.          0.80115264  2.40345791 -2.40345791]
 [ 0.          0.80115264 -2.40345791 -2.40345791  1.60230527]]


In [49]:
def leapfrog_2D(N_md, p_0, phi_0, J,x,y, N): 
    epsilon = 0.1/N_md
    p = p_0
    phi = phi_0
    #phi += phi_dot*epsilon/2
    for i in range(N_md-1):
        p += P_dot_i_2D(1,L1, x,y)*epsilon
        #phi += phi_dot*epsilon
    
    p +=  P_dot_i_2D(1,L1, x,y)*epsilon/2
    #phi -= phi_dot*epsilon/2
    return p, phi

In [50]:
# # Defining initial values
# N_md = 4
# N = 20
# p_0 = 0.1
# phi_0 = 0.1
# J = 1/N       # J should be greater than 0
# # N = 15        # N range is given between 5 to 20

In [51]:
# diff = []
# range_1 = range(1,100)

# for N_md in range_1:
#     arr = leapfrog_2D(N_md, p_0, phi_0, J,x,y, N)
#     p_f = arr[0]          # We need final values of p_f and phi_f to calculate H_f
#     phi_f = arr[1]
#     H_f = Hamiltonian2D(J, L, x , y)(p_f, phi_f, J, beta, h, N)
#     H_0 = Hamiltonian2D(J, L, x , y)(p_0, phi_0, J, beta, h, N)
    
# # The following is the quantity we want on y-axis
#     diff.append(abs((H_f-H_0)/H_0)) 

In [6]:
# # Now we code the HMC algorithm

# # Understanding: We sample from N space and integrate the EOM's using leapfrog to get p' and phi'
# # Then we accept the value of phi' with the Metropolis accept/reject (with the given prob.) and repeat the process
# # To keep in mind: We store every value of phi regardless to generate our Markov chain. If the algorithm rejects phi' we
# # store phi

# # So we need to accept and reject according to the probability distribution given in the sheet which is basically exp(H)

# # Size of ensemble is given as N_cfg

# def markov_chain(phi_i,N_cfg,N_md,J,N):
#     chain = []
#     chain.append(phi_i)
#     for i in range(N_cfg):
#         p = np.random.normal(0,1)  # Random sample
#         new = leapfrog(N_md, p, chain[-1], J, beta, h, N) # Integrating leapfrog algorithm
#         prob = np.exp(hamiltonian(p,chain[-1], J, beta, h, N)-hamiltonian(new[0], new[1], J, beta, h, N))
# #  Given probability distribution function defined above
#         if prob > np.random.uniform(0,1):   # Accept/reject
#             chain.append(new[1])  
#         else:                           
#             chain.append(chain[-1]) 
#     return chain